In [40]:
import nltk
import pandas as pd
import plotly.figure_factory as ff
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [2]:
# import data 
topic_df = pd.read_csv('papers_bertopics_multi.csv', index_col=0)
topic_df.reset_index(inplace=True, drop=True)
topic_df.fillna(';', inplace=True)
topic_df['keywords'] = topic_df['Index Keywords'] +' '+ topic_df['Author Keywords'] +' '+ topic_df['topic_words_0'] +' '+ topic_df['topic_words_1'] +' '+ topic_df['topic_words_2'] +' '+ topic_df['topic_words_3'] +' '+ topic_df['topic_words_4']
topic_df['keywords'] = topic_df['keywords'].str.replace(';', '')

### Keyword graph creation

In [3]:
topic_kw_df = topic_df[topic_df['keywords'].notna()].reset_index(drop=True)

In [4]:
# Count Vectorizer
vect = CountVectorizer()
vects = vect.fit_transform(topic_df['keywords'])
term_doc_df = pd.DataFrame(vects.todense()) # create term-doc freqeuncy table
term_doc_df.columns = vect.get_feature_names_out() # add words as labels
term_doc_df = term_doc_df>0

In [6]:
# make term pairwise presence matrix
df_pairwise = pd.DataFrame(0, index=term_doc_df.columns, columns=term_doc_df.columns)
win_size = 50
term_num = len(term_doc_df)
start_point = 0 # default to 0
# select_mat_sum_ar = np.zeros(term_num,term_num)
for i in range(start_point, term_num, win_size):
    print('start', i)
    select_df = term_doc_df.iloc[i:i+win_size]
    df_pairwise = df_pairwise + sum(select_df.apply(lambda x: (np.array([x], dtype=int).T@np.array([x], dtype=int)), axis=1))
    print('end', i, '\n')
df_pairwise.to_csv('keywords_co_ocurrence_matrix.csv')

start 0
end 0 

start 50
end 50 

start 100
end 100 

start 150
end 150 

start 200
end 200 

start 250
end 250 

start 300
end 300 

start 350
end 350 

start 400
end 400 

start 450
end 450 

start 500
end 500 

start 550
end 550 

start 600
end 600 

start 650
end 650 

start 700
end 700 

start 750
end 750 

start 800
end 800 

start 850
end 850 

start 900
end 900 

start 950
end 950 

start 1000
end 1000 

start 1050
end 1050 

start 1100
end 1100 

start 1150
end 1150 

start 1200
end 1200 

start 1250
end 1250 

start 1300
end 1300 

start 1350
end 1350 

start 1400
end 1400 

start 1450
end 1450 

start 1500
end 1500 

start 1550
end 1550 

start 1600
end 1600 

start 1650
end 1650 

start 1700
end 1700 

start 1750
end 1750 

start 1800
end 1800 

start 1850
end 1850 

start 1900
end 1900 

start 1950
end 1950 

start 2000
end 2000 

start 2050
end 2050 

start 2100
end 2100 

start 2150
end 2150 

start 2200
end 2200 

start 2250
end 2250 

start 2300
end 2300 

start 2350
e

In [52]:
# make matrix to edgelist
pair_upper_df = pd.DataFrame(np.triu(df_pairwise, 1), index=df_pairwise.index, columns=df_pairwise.columns).stack().reset_index()
pair_upper_df.columns = ['head_node', 'tail_node', 'weight']
pair_upper_df = pair_upper_df[pair_upper_df['weight']>1].reset_index(drop=True)
# filter out stopwords
nltk.download('stopwords')
stopwords_lst = list(set(stopwords.words('english')))
head_filt_df = pair_upper_df[~pair_upper_df['head_node'].isin(stopwords_lst)]
pair_upper_filt_df = head_filt_df[~head_filt_df['tail_node'].isin(stopwords_lst)]
pair_upper_filt_df = pair_upper_filt_df.sort_values('weight', ascending=False).reset_index(drop=True)
pair_upper_filt_df.to_csv('keywords_edgelist.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GCM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
pair_upper_filt_df

head_node    tail_node  weight
0           beetle   coleoptera    1194
1       coleoptera   scolytinae     956
2           beetle      species     793
3           beetle   scolytinae     774
4       coleoptera      species     774
...            ...          ...     ...
197875    dieldrin       hampei       2
197876    dieldrin     genetics       2
197877        diel      species       2
197878        diel     resource       2
197879      yunnan  yunnanensis       2

[197880 rows x 3 columns]